In [45]:
import json
import os
import sklearn
import numpy
import xgboost as xgb
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from collections import namedtuple
import sys
from math import log,tan,pi,radians
import pandas as pd
from sodapy import Socrata
from config import CHI_API_KEY
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [46]:
df = pd.read_csv("hoods.csv")

In [47]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [65]:
df = df[df.hood_label != 0]
df.describe

<bound method NDFrame.describe of                                           crash_record_id  \
0       c866f3095932bfe799226e255e1079fc2bbc49bb8fe376...   
1       725c92ece43c197a378a4f774e50ad15a4c25360ad5045...   
2       71b14bfa4ef2da35945350552503128df47a45952f0805...   
3       79c45fc1ecbe1902dc6cef723573607eeca83d33a5d329...   
4       49269ed31a112084261ca43ff3934714374c1e6aa4c36b...   
...                                                   ...   
453925  964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...   
453926  1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...   
453927  957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...   
453928  f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...   
453929  a802658be15312809c771559e4f81088cfb226830792a5...   

                     crash_date  posted_speed_limit traffic_control_device  \
0       2020-11-19T00:25:00.000                  30         TRAFFIC SIGNAL   
1       2020-11-18T23:00:00.000                  30            NO CONTROLS   

In [68]:
df = df[df['hood_label'].notna()]
df = df[df['weather_condition'].notna()]
df = df[df['lighting_condition'].notna()]
df = df[df['crash_day_of_week'].notna()]
df.dropna(subset = ['hood_label', 'weather_condition', 'lighting_condition', 'crash_day_of_week', 'damage'], inplace=True)

In [69]:
df['crash_day_of_week'] = df['crash_day_of_week'].replace([1,2,3,4,5,6,7],['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'])

In [81]:
df['crash_day_of_week'].value_counts()

Friday       72489
Saturday     65749
Tuesday      64135
Thursday     63948
Wednesday    63451
Monday       62026
Sunday       54058
Name: crash_day_of_week, dtype: int64

In [80]:
df['crash_hour'] = df['crash_hour'].replace([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],['Midnight', '1am', '2am', '3am', '4am', '5am', '6am', '7am', '8am', '9am', '10am', '11am', 'Noon', '1pm', '2pm', '3pm', '4pm',
                                                                                                                    '5pm', '6pm', '7pm', '8pm', '9pm', '10pm', '11pm'])
df['crash_hour'].value_counts()

3pm         34157
2pm         34048
4pm         33607
1pm         30216
5pm         27730
Noon        27377
11am        26342
7am         24257
10am        22650
8am         20999
9am         20436
6pm         20204
6am         19669
7pm         15960
8pm         14288
9pm         13160
10pm        11028
5am         10081
0            8938
Midnight     7583
1am          6613
4am          6114
2am          5401
3am          4998
Name: crash_hour, dtype: int64

In [82]:
X = pd.get_dummies(df[['hood_label', "weather_condition", "lighting_condition", "crash_day_of_week", "crash_hour"]])

In [83]:
values = array(df["hood_label"])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
hl_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [84]:
values = array(df["damage"])

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
damage_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [85]:
y = damage_onehot_encoded
print(X.shape)
print(y.shape)

(445856, 125)
(445856, 3)


In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [87]:
xgb_estimator = xgb.XGBClassifier(objective='binary:logistic')

In [88]:
multilabel_model = MultiOutputClassifier(xgb_estimator)

In [89]:
multilabel_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None, gamma=None,
                                              gpu_id=None,
                                              importance_type='gain',
                                              interaction_constraints=None,
                                              learning_rate=None,
                                              max_delta_step=None,
                                              max_depth=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                         

In [90]:
print('Accuracy on test data: {:.1f}%'.format(accuracy_score(y_test, multilabel_model.predict(X_test))*100))

Accuracy on test data: 51.0%


In [92]:
from joblib import dump, load
dump(multilabel_model, 'damage.joblib') 

['damage.joblib']

In [94]:
test = X.iloc[3:5]
test

,hood_label_ANDERSONVILLE,"hood_label_ARCHER HEIGHTS,WEST ELSDON","hood_label_ARMOUR SQUARE,CHINATOWN",hood_label_ASHBURN,hood_label_AUBURN GRESHAM,hood_label_AUSTIN,"hood_label_AVALON PARK,CALUMET HEIGHTS",hood_label_BACK OF THE YARDS,"hood_label_BELMONT CRAGIN,HERMOSA","hood_label_BELMONT CRAIGIN,HERMOSA",...,crash_hour_6am,crash_hour_6pm,crash_hour_7am,crash_hour_7pm,crash_hour_8am,crash_hour_8pm,crash_hour_9am,crash_hour_9pm,crash_hour_Midnight,crash_hour_Noon
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [95]:
damage = load('damage.joblib') 

In [96]:
damage.predict_proba(test)

[array([[0.8925331 , 0.10746686],
        [0.88090926, 0.11909076]], dtype=float32),
 array([[0.7185876 , 0.28141242],
        [0.7550169 , 0.24498306]], dtype=float32),
 array([[0.39352572, 0.6064743 ],
        [0.37625992, 0.6237401 ]], dtype=float32)]

In [98]:
test.to_csv('test.csv')